In [1]:
import copy
import ff
import dmrs_functions
from collections import namedtuple, defaultdict
import goal_updates
import prob_model
import pddl_functions
import numpy as np
import multiprocessing as mp
import time
from ff import NoPlanError, IDontKnowWhatIsGoingOnError
import logging
import world
import os
import torch

In [2]:
problem_dir = 'onerule'
problems = os.listdir(problem_dir)
w = world.PDDLWorld('blocks-domain.pddl', '{}/{}'.format(problem_dir, problems[0]))

Parsing domain blocks-domain.pddl
Parsing problem onerule/problem1.pddl


In [9]:

w.update(*put_on_top('b0', w))
w.update(*put_on_top('b2', w))
obs = w.sense()

In [6]:
def get_top(relations):
    stuff = list(filter(lambda x: 'in-tower' in x[1], relations.items()))
    if len(stuff) == 1:
        return stuff[0][0]
    stuff = dict(stuff)
    bottom = 't0'
    while True:
        on = stuff.pop(bottom)['on']
        bottom = on.args.args[0].arg_name
        if len(stuff) == 1:
            return bottom



In [8]:
def put_on_top(obj, w):
    obs = w.sense()
    top = get_top(obs.relations)
    return 'put', [obj, top]

In [5]:
def select_action(action_id, w):
    obs = w.sense()
    objs = obs.objects
    obj = objs[action_id]
    return put_on_top(obj, obs)

In [34]:
put_on_top('b2', w)

('put', ['t0', 'b2'])

In [10]:
def objects_not_in_tower(w):
    observation = w.sense()
    objects = observation.objects
    rels = observation.relations
    
    out_objects = []
    
    for o in objects:
        if 'in-tower' not in rels[o]:
            out_objects.append(o)
    return out_objects
objects_not_in_tower(w)

['b1', 'b3', 'b4', 'b5', 'b6', 'b7', 'b8', 'b9']

In [15]:
def get_state(o1, o2, obs):
    return np.concatenate([obs.colours[o1], obs.colours[o2]])
get_state('b1', 'b2', w.sense())

array([1.        , 0.        , 0.        , 1.        , 0.64705884,
       0.        ], dtype=float32)